In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import helper
import fc_model

## Load data

In [3]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

0it [00:00, ?it/s]

26427392it [00:04, 6489773.26it/s]                              


Extracting /home/muha_muttaqien/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/muha_muttaqien/.pytorch/F_MNIST_data/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 43204.16it/s]                           
0it [00:00, ?it/s]

Extracting /home/muha_muttaqien/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/muha_muttaqien/.pytorch/F_MNIST_data/FashionMNIST/raw


4423680it [00:02, 1944005.04it/s]                            
0it [00:00, ?it/s]

Extracting /home/muha_muttaqien/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/muha_muttaqien/.pytorch/F_MNIST_data/FashionMNIST/raw


8192it [00:00, 17122.76it/s]            

Extracting /home/muha_muttaqien/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/muha_muttaqien/.pytorch/F_MNIST_data/FashionMNIST/raw
Processing...
Done!


## Train the network

In [4]:
# create the network, define the criterion and optimizer
model = fc_model.Network(input_size=784, output_size=10, hidden_layers=[512, 256, 128])
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
fc_model.train(model, trainloader, testloader, criterion, optimizer, epochs=2)

Epoch: 1/2..  Training Loss: 1.678..  Test Loss: 0.929..  Test Accuracy: 0.690
Epoch: 1/2..  Training Loss: 0.980..  Test Loss: 0.746..  Test Accuracy: 0.728
Epoch: 1/2..  Training Loss: 0.880..  Test Loss: 0.703..  Test Accuracy: 0.724
Epoch: 1/2..  Training Loss: 0.822..  Test Loss: 0.652..  Test Accuracy: 0.743
Epoch: 1/2..  Training Loss: 0.757..  Test Loss: 0.618..  Test Accuracy: 0.773
Epoch: 1/2..  Training Loss: 0.716..  Test Loss: 0.582..  Test Accuracy: 0.789
Epoch: 1/2..  Training Loss: 0.701..  Test Loss: 0.586..  Test Accuracy: 0.783
Epoch: 1/2..  Training Loss: 0.697..  Test Loss: 0.562..  Test Accuracy: 0.791
Epoch: 1/2..  Training Loss: 0.622..  Test Loss: 0.556..  Test Accuracy: 0.788
Epoch: 1/2..  Training Loss: 0.685..  Test Loss: 0.535..  Test Accuracy: 0.805
Epoch: 1/2..  Training Loss: 0.634..  Test Loss: 0.516..  Test Accuracy: 0.810
Epoch: 1/2..  Training Loss: 0.654..  Test Loss: 0.526..  Test Accuracy: 0.808
Epoch: 1/2..  Training Loss: 0.630..  Test Loss: 0.5

## Save the networks

In [7]:
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

Our model: 

 Network(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
  )
  (output): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
) 

The state dict keys: 

 odict_keys(['hidden_layers.0.weight', 'hidden_layers.0.bias', 'hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'output.weight', 'output.bias'])


In [8]:
torch.save(model.state_dict(), 'fc_model.pth')

## Load the networks

In [9]:
state_dict = torch.load('fc_model.pth')
print(state_dict.keys())

odict_keys(['hidden_layers.0.weight', 'hidden_layers.0.bias', 'hidden_layers.1.weight', 'hidden_layers.1.bias', 'hidden_layers.2.weight', 'hidden_layers.2.bias', 'output.weight', 'output.bias'])


In [10]:
state_dict['hidden_layers.0.weight']

tensor([[ 0.0295,  0.0177,  0.0415,  ...,  0.0310,  0.0320,  0.0608],
        [ 0.0024, -0.0032,  0.0345,  ..., -0.0231,  0.0080,  0.0324],
        [ 0.0141,  0.0152,  0.0133,  ...,  0.0133, -0.0291, -0.0164],
        ...,
        [ 0.0160,  0.0367, -0.0152,  ...,  0.0294, -0.0033,  0.0449],
        [ 0.0356,  0.0085,  0.0404,  ...,  0.0158, -0.0258,  0.0021],
        [ 0.0488,  0.0098,  0.0432,  ...,  0.0354,  0.0472,  0.0124]])

In [11]:
model.load_state_dict(state_dict)

<All keys matched successfully>

## Inform the networks

In [16]:
checkpoint = {'input_size': 784,
              'output_size': 10,
              'hidden_layers': [each.out_features for each in model.hidden_layers],
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'fc_model.pth')

In [17]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

In [18]:
model = load_checkpoint('fc_model.pth')
print(model)

Network(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
  )
  (output): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


<hr/>